<a href="https://colab.research.google.com/github/NIKKS1234/emotion_to_emoji/blob/main/openCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#pip install flask
#uncomment the above line to install flask framework

In [ ]:
#importing all the important libraries for our code
from flask import Flask, render_template, Response
import tkinter as tk
from tkinter import *
from PIL import Image, ImageTk
import numpy as np
import random 
import matplotlib.pyplot as plt
import os
import glob as gb
import cv2
import seaborn as sns
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Input,Dropout,GlobalAveragePooling2D,Flatten,Conv2D,BatchNormalization,Activation,MaxPooling2D
from keras.models import Model,Sequential
from tensorflow.keras.optimizers import Adam,SGD,RMSprop
import time
from tensorflow.keras.models import load_model

In [ ]:
#loading our saved model
emotion_model = load_model(r'
# ADD PATH OF SAVED MODEL 'research50.h5' AFTER DOWNLOADING
')
# if you want vgg16 model then uncomment below 
#emotion_model = load_model(r'C:\Users\nikks\Downloads\model.h5')

In [ ]:
# initializing flask application
app=Flask(__name__)

# getting camera view
cap=cv2.VideoCapture(0)

In [ ]:
#defining function that returns stream of bytes of frame to flask call
def gen():
    cv2.ocl.setUseOpenCL(False)
    #emotion dictionary with key as the last layer of deep neural network, and value as the corresponding emotion
    #emotion_dict={0:"Angry", 1:"Disgusted", 2:"Fearful", 3:"Happy", 4:"Neutral", 5:"Sad", 6:"Surprised"}

    #UPDATE PATH OF EMOJIS AFTER DOWNLOADING THE EMOJI FOLDER

    emotion_dict={0:r"\angry.jpg", 1:r"\disgusted.jpg", 2:r"\fearful.jpg", 3:r"\happy.jpg", 4:r"\neutral.jpg", 5:r"\sad.jpg", 6:r"\surprise.jpg"}

    while True:
        ret, frame= cap.read()
        if not ret:
            break

            
        bounding_box= cv2.CascadeClassifier(r'C:\Python310\Lib\site-packages\cv2\data\haarcascade_frontalface_default.xml')
        gray_frame=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        num_faces=bounding_box.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)
        for (x, y, w, h) in num_faces:
            cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
            roi_gray_frame=gray_frame[y:y+h, x:x+w]
            cropped_img=np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48,48)), -1), 0)

            #getting array of seven values, each value representing relative probability of a particular emotion of being the result
            emotion_prediction=emotion_model.predict(cropped_img)

            #getting the index of maximum probable emotion
            maxindex=int(np.argmax(emotion_prediction))

            #reading the emoji corresponding to maxindex from emotion dictionary
            em_img=cv2.imread(emotion_dict[maxindex])

            #getting dimensions of image and adding to the video frame
            img_height,img_w,_=em_img.shape
            y=y-50
            frame[y:y+img_height,x:x+img_w]=em_img

            #breking the video input frame to bytes of information to be returned to flask call
            ret,buffer=cv2.imencode('.jpg',frame)
            frame=buffer.tobytes()
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')
            # to get stable result giving a time lag of .1 second
            time.sleep(.1)

In [ ]:
@app.route('/')
def home():
  #error handling if the homepage of html file is not shown up
    try:
        return render_template(r'
              #  ADD PATH OF 'main.html' FILE AFTER DOWNLOADING
        ')#render template helps in showing a separate html file in flask
    except Exception as e:
        return str(e)
@app.route('/d')
def d():
    return Response(gen(), mimetype='multipart/x-mixed-replace; boundary=frame')# Response helps in returning stream of bytes of frame
if __name__=="__main__":
  #running application
    Flask.run(app)

In [ ]:
#code to check emotion to text

#  cv2.ocl.setUseOpenCL(False)

# emotion_dict={0:"Angry", 1:"Disgusted", 2:"Fearful", 3:"Happy", 4:"Neutral", 5:"Sad", 6:"Surprised"}

#  cap=cv2.VideoCapture(0)

#  while True:
#      ret, frame= cap.read()
#      if not ret:
#          break
#      bounding_box= cv2.CascadeClassifier(r'C:\Python310\Lib\site-packages\cv2\data\haarcascade_frontalface_default.xml')
#      gray_frame=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#      num_faces=bounding_box.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)
#      for (x, y, w, h) in num_faces:
#          cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
#          roi_gray_frame=gray_frame[y:y+h, x:x+w]
#          cropped_img=np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48,48)), -1), 0)
#          emotion_prediction=emotion_model.predict(cropped_img)
#          maxindex=int(np.argmax(emotion_prediction))
#          cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
#      cv2.imshow('Video', cv2.resize(frame, (1200, 860), interpolation=cv2.INTER_CUBIC))
#      if cv2.waitKey(1)&0xFF==ord('q'):
#          cap.release()
#          cv2.destroyAllWindows()
#          break;